In [2]:
import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data

np.random.seed(1)

In [11]:
mnist = input_data.read_data_sets("MNIST_data/",one_hot=True)
train_X, train_Y, test_X, test_Y = mnist.train.images, mnist.train.labels, mnist.test.images, mnist.test.labels

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [4]:
print("Train X shape",train_X.shape)
print("Train Y shape",train_Y.shape)
print("Test X shape",test_X.shape)
print("Test Y shape",test_Y.shape)

Train X shape (55000, 784)
Train Y shape (55000, 10)
Test X shape (10000, 784)
Test Y shape (10000, 10)


In [5]:
number_features = train_X.shape[1]
number_output_classes = train_Y.shape[1]

In [6]:
def weight_init(shape):
    return tf.Variable(tf.random_normal(shape))

In [7]:
def model(X, W1, b1, W2, b2, p_keep_input, p_keep_hidden):
    X = tf.nn.dropout(X,keep_prob=p_keep_input)
    hidden_layer = tf.nn.relu(tf.add(tf.matmul(X, W1), b1))
    hidden_layer = tf.nn.dropout(hidden_layer,keep_prob=p_keep_hidden)
    y_estimated = tf.sigmoid(tf.add(tf.matmul(hidden_layer, W2), b2))
    return y_estimated

In [8]:
number_node = 232
p_keep_input = tf.placeholder(tf.float32)
p_keep_hidden = tf.placeholder(tf.float32)
X = tf.placeholder(tf.float32,shape=(None,train_X.shape[1]))
Y = tf.placeholder(tf.float32,shape=(None,train_Y.shape[1]))

In [9]:
W1 = weight_init([number_features,number_node])
b1 = weight_init([number_node,])
W2 = weight_init([number_node,number_output_classes])
b2 = weight_init([number_output_classes,])

In [12]:
y_hat = model(X,W1,b1,W2,b2,p_keep_input,p_keep_hidden)
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=y_hat,labels=Y))
train_step = tf.train.RMSPropOptimizer(0.001, 0.9).minimize(cost)
predict_step = tf.argmax(y_hat, 1)

In [ ]:
with tf.Session() as sess:
    tf.global_variables_initializer().run()
    
    for i in range(1000):
        batch_x, batch_y = mnist.train.next_batch(100)
        sess.run(train_step,feed_dict={X:batch_x, Y:batch_y,p_keep_input:0.8,p_keep_hidden:0.5})
        print(i,"th iteration, accuracy is ",np.mean(np.argmax(test_Y, axis=1) ==
                         sess.run(predict_step, feed_dict={X: test_X,
                                                           p_keep_input:1.0,
                                                           p_keep_hidden:1.0})))